In [ ]:
# External package imports
import tensorflow as tf
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from data_loader import initialize_dataset

import keras
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD

from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/midterm_project/cassava-leaf-disease-classification/train_tfrecords2"

In [ ]:
# Get a list of the training record files
tfrecord_files = glob.glob(os.path.join(path, "*.tfrec"))

# Define the number of files for training, 70% of files
n_file_train = int(0.7 * len(tfrecord_files))

# Split the record files into 70% training and 30% validation
tfrecord_files_train = tfrecord_files[:n_file_train]
tfrecord_files_valid = tfrecord_files[n_file_train:]

In [ ]:
# Define batch size
BATCH_SIZE = 32

# Define processed image size
HEIGHT = 258
WIDTH = HEIGHT
#IMG_SIZE = (HEIGHT, WIDTH)

In [ ]:
# Initialize the training data loader, with random flipping and rotations
dataset_train = initialize_dataset(tfrecord_files_train, batch_size=BATCH_SIZE, img_size=(HEIGHT, WIDTH), flip=True, rot=(-1, 1))

# Initialize the validation loader, with no flipping or rotations
dataset_valid = initialize_dataset(tfrecord_files_valid, img_size=(HEIGHT, WIDTH))

In [ ]:
initial_learning_rate = 0.00002
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9)
opt = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

early_stop = EarlyStopping(monitor='val_accuracy', mode='max', 
                       patience=4, restore_best_weights=True, verbose=1)

def build_ResNet():
    base_model = tf.keras.applications.ResNet101(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(WIDTH, HEIGHT, 3),
        pooling=None,
        classes=1000
    )
    #base_model.trainable = False
    x = base_model.output
    model = Flatten()(x)
    model = Dropout(rate=0.25)(model)
    predictions = Dense(5, activation = "softmax")(model)
    model = Model(inputs = base_model.input, outputs = predictions)
    return model
ResNet = build_ResNet()
ResNet.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_hist = ResNet.fit(
    x=dataset_train,
    epochs=6,
    validation_data=dataset_valid,
    callbacks = [early_stop]
)

171450368/171446536 [==============================] - 1s 0us/step
Epoch 1/6
460/460 [==============================] - 588s 1s/step - loss: 1.4818 - accuracy: 0.6024 - val_loss: 1.8458 - val_accuracy: 0.7148
Epoch 2/6
460/460 [==============================] - 557s 1s/step - loss: 0.9010 - accuracy: 0.7205 - val_loss: 1.4153 - val_accuracy: 0.7534
Epoch 3/6
460/460 [==============================] - 558s 1s/step - loss: 0.7309 - accuracy: 0.7642 - val_loss: 1.0980 - val_accuracy: 0.7766
Epoch 4/6
460/460 [==============================] - 559s 1s/step - loss: 0.6439 - accuracy: 0.7858 - val_loss: 1.2291 - val_accuracy: 0.7875
Epoch 5/6
460/460 [==============================] - 559s 1s/step - loss: 0.5833 - accuracy: 0.8009 - val_loss: 1.3404 - val_accuracy: 0.7850
Epoch 6/6
460/460 [==============================] - 558s 1s/step - loss: 0.5465 - accuracy: 0.8166 - val_loss: 0.9866 - val_accuracy: 0.8115


In [ ]:
# This model was trained on 96x96 images

initial_learning_rate = 0.00004
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9)
   # staircase=True)
opt = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

early_stop = EarlyStopping(monitor='val_accuracy', mode='max', 
                       patience=4, restore_best_weights=True, verbose=1)
  
def build_IncepNet():
    base_model =   tf.keras.applications.InceptionResNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(WIDTH, HEIGHT, 3),
        pooling=None,
      )
    #base_model.trainable = False
    x = base_model.output
    model = Flatten()(x)
    #model = Dense(128, activation = "relu")(model)
    model = Dropout(rate=0.25)(model)
    predictions = Dense(5, activation = "softmax")(model)
    model = Model(inputs = base_model.input, outputs = predictions)
    return model
IncepNet = build_IncepNet()
IncepNet.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_hist = IncepNet.fit(
    x=dataset_train,
    epochs=6,
    validation_data=dataset_valid,
    callbacks = [early_stop]
)

219062272/219055592 [==============================] - 2s 0us/step
Epoch 1/6
460/460 [==============================] - 655s 1s/step - loss: 1.0157 - accuracy: 0.6386 - val_loss: 1.3324 - val_accuracy: 0.7513
Epoch 2/6
460/460 [==============================] - 614s 1s/step - loss: 0.6867 - accuracy: 0.7512 - val_loss: 1.5809 - val_accuracy: 0.7664
Epoch 3/6
460/460 [==============================] - 614s 1s/step - loss: 0.5752 - accuracy: 0.7935 - val_loss: 1.0000 - val_accuracy: 0.8015
Epoch 4/6
460/460 [==============================] - 617s 1s/step - loss: 0.4993 - accuracy: 0.8236 - val_loss: 1.2758 - val_accuracy: 0.8121
Epoch 5/6
460/460 [==============================] - 616s 1s/step - loss: 0.4390 - accuracy: 0.8465 - val_loss: 0.9979 - val_accuracy: 0.8218
Epoch 6/6
460/460 [==============================] - 614s 1s/step - loss: 0.4041 - accuracy: 0.8559 - val_loss: 1.5922 - val_accuracy: 0.8106


In [1]:
#imports
import tensorflow as tf

# setup tensorboard, directories
!rm -rf ./logs
!mkdir ./logs/
!mkdir ./logs/EffNet

log_dir="./logs/EffNet/"
def tensorboard_callback(exp_name):
  return tf.keras.callbacks.TensorBoard(log_dir=log_dir + exp_name, profile_batch=0, histogram_freq=1)
# launch tensorboard with specific directory
%load_ext tensorboard
%tensorboard --logdir logs/EffNet

C:\Users\nclan\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nclan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\nclan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
'rm' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.
The syntax of the command is incorrect.


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 7536.

In [ ]:
# Some high level ideas taken from https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-tensorflow-training
initial_learning_rate = 0.00004
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9)
opt = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

early_stop = EarlyStopping(monitor='val_accuracy', mode='max', 
                       patience=4, restore_best_weights=True, verbose=1)
  
def build_EffNet():
    base_model = tf.keras.applications.EfficientNetB1(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(HEIGHT, WIDTH, 3),
        pooling=None,
      )
    #base_model.trainable = False
    x = base_model.output
    model = Flatten()(x)
    model = Dropout(rate=0.25)(model)
    predictions = Dense(5, activation = "softmax")(model)
    model = Model(inputs = base_model.input, outputs = predictions)
    return model
EffNet = build_EffNet()
EffNet.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_hist = EffNet.fit(
    x=dataset_train,
    epochs=10,
    validation_data=dataset_valid,
    callbacks = [early_stop]
)

27025408/27018416 [==============================] - 0s 0us/step
Epoch 1/10
460/460 [==============================] - 502s 982ms/step - loss: 1.1646 - accuracy: 0.6143 - val_loss: 0.7691 - val_accuracy: 0.7341
Epoch 2/10
460/460 [==============================] - 439s 953ms/step - loss: 0.7820 - accuracy: 0.7373 - val_loss: 0.7293 - val_accuracy: 0.7537
Epoch 3/10
460/460 [==============================] - 440s 956ms/step - loss: 0.6978 - accuracy: 0.7632 - val_loss: 0.6389 - val_accuracy: 0.7837
Epoch 4/10
460/460 [==============================] - 442s 961ms/step - loss: 0.6071 - accuracy: 0.7883 - val_loss: 0.6299 - val_accuracy: 0.7899
Epoch 5/10
460/460 [==============================] - 440s 955ms/step - loss: 0.5708 - accuracy: 0.8032 - val_loss: 0.5960 - val_accuracy: 0.8037
Epoch 6/10
460/460 [==============================] - 438s 951ms/step - loss: 0.5339 - accuracy: 0.8196 - val_loss: 0.6064 - val_accuracy: 0.8016
Epoch 7/10
460/460 [==============================] - 440s 